In [2]:

import os
import getpass

os.environ["XAI_API_KEY"] = getpass.getpass("Enter your XAI API Key:")

In [4]:
from langchain_xai import ChatXAI

llm = ChatXAI(model="grok-3-latest")

# Using invoke (synchronous)
# response = chat.invoke("Tell me about Grok 3")

# Using stream (asynchronous)
# for chunk in llm.stream("What is 2 + 2?"):
#     print(chunk.content, end="", flush=True)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        ("user", "{input}"),
    ]
)

chain = prompt | llm

for chunk in chain.stream({"input": "Did people generally like John Wick?"}):
    print(chunk.content, end="", flush=True)

Yes, people generally liked *John Wick*. The 2014 action film, starring Keanu Reeves, received widespread praise from both audiences and critics. It holds an 86% approval rating on Rotten Tomatoes based on critic reviews and an 85% audience score, indicating strong positive reception. Fans often highlight the movie's stylish action choreography, Reeves' intense performance, and the unique world-building around the assassin underworld. On platforms like IMDb, it has a rating of 7.4/10 based on over 700,000 user votes, further showing its popularity.

The film also gained a cult following, spawning three sequels (with a fourth released in 2023) and a prequel series, *The Continental*. Social media and online discussions often reflect enthusiasm for the character and the franchise, with many appreciating its emotional depth (centered on Wick's grief) alongside the high-octane action. However, some viewers criticize the plot for being thin or overly simplistic, focusing more on revenge tha

In [8]:
from langchain.chat_models.base import init_chat_model

grok = init_chat_model(
    model="grok-3-latest",
    temperature=0.0,
    model_provider="xai",
)

response = grok.invoke("What is the capital of France?")
print(response.content)

The capital of France is Paris.


In [9]:
# read files from current directory
def read_files_in_directory(directory):
    files = []
    for filename in os.listdir(directory):
        if filename.endswith(".csv"):
            with open(os.path.join(directory, filename), "r") as file:
                files.append(file.read())
    return files


# read files from current directory
docs = read_files_in_directory(".")


In [10]:
for doc in docs:
    print(len(doc))

14747
13884
15109
19627


In [11]:
from langchain_community.document_loaders.csv_loader import CSVLoader
from datetime import datetime, timedelta

documents = []

for i in range(1, 5):
  loader = CSVLoader(
      file_path=f"john_wick_{i}.csv",
      metadata_columns=["Review_Date", "Review_Title", "Review_Url", "Author", "Rating"]
  )

  movie_docs = loader.load()
  for doc in movie_docs:

    # Add the "Movie Title" (John Wick 1, 2, ...)
    doc.metadata["Movie_Title"] = f"John Wick {i}"

    # convert "Rating" to an `int`, if no rating is provided - assume 0 rating
    doc.metadata["Rating"] = int(doc.metadata["Rating"]) if doc.metadata["Rating"] else 0

    # newer movies have a more recent "last_accessed_at"
    doc.metadata["last_accessed_at"] = datetime.now() - timedelta(days=4-i)

  documents.extend(movie_docs)

In [13]:
documents[0]

Document(metadata={'source': 'john_wick_1.csv', 'row': 0, 'Review_Date': '6 May 2015', 'Review_Title': ' Kinetic, concise, and stylish; John Wick kicks ass.\n', 'Review_Url': '/review/rw3233896/?ref_=tt_urv', 'Author': 'lnvicta', 'Rating': 8, 'Movie_Title': 'John Wick 1', 'last_accessed_at': datetime.datetime(2025, 5, 14, 19, 22, 36, 57929)}, page_content=": 0\nReview: The best way I can describe John Wick is to picture Taken but instead of Liam Neeson it's Keanu Reeves and instead of his daughter it's his dog. That's essentially the plot of the movie. John Wick (Reeves) is out to seek revenge on the people who took something he loved from him. It's a beautifully simple premise for an action movie - when action movies get convoluted, they get bad i.e. A Good Day to Die Hard. John Wick gives the viewers what they want: Awesome action, stylish stunts, kinetic chaos, and a relatable hero to tie it all together. John Wick succeeds in its simplicity.")

In [15]:
from ragas.run_config import RunConfig
from ragas.testset import TestsetGenerator
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper
# from langchain_openai import ChatOpenAI
# from langchain_openai import OpenAIEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings

# Create a RunConfig with rate limit settings
my_run_config = RunConfig(
    max_workers=8,      # Control concurrent requests (default is 16)
    timeout=180,         # Maximum time to wait for a single operation (default is 180s)
    max_retries=10,      # Maximum number of retry attempts (default is 10)
    max_wait=120,         # Maximum wait time between retries (default is 60s)
    exception_types=(Exception,)  # Types of exceptions to retry on
)

# Initialize the generator with the LLM and embedding model
generator_llm = LangchainLLMWrapper(grok)
generator_embeddings = LangchainEmbeddingsWrapper(HuggingFaceEmbeddings(model_name="Snowflake/snowflake-arctic-embed-l"))

# Set the run config for both LLM and embeddings
generator_llm.set_run_config(my_run_config)
generator_embeddings.set_run_config(my_run_config)

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)

# Use the run_config in your generate call
dataset = generator.generate_with_langchain_docs(
    documents=documents, 
    testset_size=10,
    run_config=my_run_config
)

Applying SummaryExtractor:   0%|          | 0/44 [00:00<?, ?it/s]

Applying CustomNodeFilter:   0%|          | 0/100 [00:00<?, ?it/s]

Node 58438866-9440-4036-bc4c-51c5cb605d2a does not have a summary. Skipping filtering.
Node 8957a221-cf9a-4d99-adde-ff6d5087476d does not have a summary. Skipping filtering.
Node 59628820-48c3-49b1-aea7-4776ff70f81d does not have a summary. Skipping filtering.
Node 49b09e6b-636d-4c1f-ae34-16585503458f does not have a summary. Skipping filtering.
Node 62897717-cfaf-4527-bab7-9ef442a4e7d3 does not have a summary. Skipping filtering.
Node 29ca929b-db3c-4c67-871e-c022a775ee6a does not have a summary. Skipping filtering.
Node 99fdbf90-6102-458c-a37b-965bc1725c17 does not have a summary. Skipping filtering.
Node 674323a8-a60d-4c4f-bfc9-34e314cd84b3 does not have a summary. Skipping filtering.
Node 4baad603-69fc-4502-9c5c-3d96610023be does not have a summary. Skipping filtering.
Node 59d92c27-fbe5-46c6-b521-6ee165e2cf42 does not have a summary. Skipping filtering.
Node 04e5a612-ce1c-4c2c-9be2-1e793b9c9292 does not have a summary. Skipping filtering.
Node 8681135c-094e-413d-a473-d22902ccc4fd d

Applying [EmbeddingExtractor, ThemesExtractor, NERExtractor]:   0%|          | 0/244 [00:00<?, ?it/s]

Applying [CosineSimilarityBuilder, OverlapScoreBuilder]:   0%|          | 0/2 [00:00<?, ?it/s]

Generating personas:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Scenarios:   0%|          | 0/3 [00:00<?, ?it/s]

Generating Samples:   0%|          | 0/12 [00:00<?, ?it/s]

In [16]:
dataset.to_pandas().to_csv("testset.csv", index=False)

In [17]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,"Hey, can u tell me what’s the deal with John W...",[: 0\nReview: The best way I can describe John...,"John Wick is basically like Taken, but with Ke...",single_hop_specifc_query_synthesizer
1,What does Cinematic Critic Carl think about Jo...,[: 2\nReview: With the fourth installment scor...,Cinematic Critic Carl mentions that with the f...,single_hop_specifc_query_synthesizer
2,What makes John Wick stand out as an action mo...,[: 3\nReview: John wick has a very simple reve...,John Wick stands out due to its virtuoso actio...,single_hop_specifc_query_synthesizer
3,What role do Russian mobsters play in John Wick?,[: 4\nReview: Though he no longer has a taste ...,"In John Wick, Russian mobsters are the antagon...",single_hop_specifc_query_synthesizer
4,"Heyy, can sum1 plz tell me how da movie revues...",[<1-hop>\n\n: 23\nReview: Rating 10/10\nI was ...,The movie reviews for 'John Wick' and 'John Wi...,multi_hop_abstract_query_synthesizer
5,"Hey, can u tell me how the reveng thriller vib...","[<1-hop>\n\n: 6\nReview: In 2014, a Keanu Reev...",The revenge thriller vibe of John Wick holds u...,multi_hop_abstract_query_synthesizer
6,How dose the portrayal of Keanu Reeves as a re...,[<1-hop>\n\n: 16\nReview: John Wick 3 is witho...,"In John Wick 3, Keanu Reeves' portrayal as a r...",multi_hop_abstract_query_synthesizer
7,How do the reviws compare the hybrid comic boo...,[<1-hop>\n\n: 9\nReview: ...all at the same ti...,The reviews present contrasting opinions on th...,multi_hop_abstract_query_synthesizer
8,Considering the evolution of the John Wick fra...,[<1-hop>\n\n: 4\nReview: I went to the cinema ...,"In 'John Wick: Chapter 2,' John, portrayed by ...",multi_hop_specific_query_synthesizer
9,What makes John Wick: Chapter 4 stand out as t...,[<1-hop>\n\n: 20\nReview: In a world where mov...,John Wick: Chapter 4 is considered the best in...,multi_hop_specific_query_synthesizer
